In [55]:
import re
from typing import List

In [15]:
def b_x(n):
    return list(bin(n).replace("0b", ""))

In [49]:
def b(n):
    return [int(e) for e in "{0:036b}".format(int(n))]

In [53]:
def b36(n):
    return list("{0:036b}".format(int(n)))

In [7]:
# [int(e) for e in list("{0:036b}".format(11))] ==  bb(11)

True

In [17]:
def bb(num):
    return list(reversed([int(bool(num & (1 << n))) for n in range(36)]))  # np.asarray(

In [18]:
b(11)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1]

In [19]:
e = """
mask = 100X100X101011111X100000100X11010011
mem[33323] = 349380
mem[52742] = 116688965
mem[4113] = 11499
mem[15819] = 313303
mem[23239] = 755579063
""".strip()

In [44]:
mask = re.search(r"mask\s=\s([X|1|0]{36})", e)

In [47]:
list(mask.groups()[0])

['1',
 '0',
 '0',
 'X',
 '1',
 '0',
 '0',
 'X',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 'X',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 'X',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1']

In [36]:
assert mask.groups()[0] == "100X100X101011111X100000100X11010011"

In [38]:
instruction = re.findall(r"mem\[(?P<position>\d+)\]\s=\s(?P<value>\d+\b)", e)

In [67]:
memory = {}

In [73]:
def apply_mask_to_num(num: str, mask: str) -> int:
    
    mask: List[str] = list(mask)
    
    num: List[str] = b36(num)
    
    result = []
    
    for n, m in zip(num, mask):
        
        if m == "X":
            result.append(n)
        else:
            result.append(m)
            
    binary_str = "".join(result)
    
    return int(binary_str, 2)

In [75]:
apply_mask_to_num('349380', mask.groups()[0])

36691642579

In [76]:
n = 349380
assert int("".join(b36(str(349380))), 2) == 349380

In [88]:
def process_group(group: str):
    # mask = re.search(r"mask\s=\s([X|1|0]{36})", group)
    mask = re.search(r"\s=\s([X|1|0]{36})", group)
    
    mask = mask.groups()[0]
    
    instructions = re.findall(r"mem\[(?P<position>\d+)\]\s=\s(?P<value>\d+\b)", group)
    
    for loc, value in instructions:
        
        memory[loc] = apply_mask_to_num(value, mask)

In [89]:
process_group(e)

In [90]:
memory

{'33323': 36691642579,
 '52742': 36691642579,
 '4113': 36691380435,
 '15819': 36691642835,
 '23239': 36691380435}

In [91]:
with open("input.txt") as f:
    data = f.read()

In [92]:
groups = data.split("mask")[1:]

In [93]:
memory = {}

In [94]:
for g in groups:
    process_group(g)

In [97]:
sum(memory.values())

13556564111697

## Part 2

In [189]:
from itertools import product, permutations

In [192]:
list(product("10", repeat=3))

[('1', '1', '1'),
 ('1', '1', '0'),
 ('1', '0', '1'),
 ('1', '0', '0'),
 ('0', '1', '1'),
 ('0', '1', '0'),
 ('0', '0', '1'),
 ('0', '0', '0')]

In [107]:
def apply_mask_to_num(num: str, mask: str) -> int:

    mask: List[str] = list(mask)

    num: List[str] = b36(num)

    result = []

    for n, m in zip(num, mask):

        if m == "0":
            result.append(n)
        else:
            result.append(m)

    binary_str = "".join(result)
    
    return binary_str

    return int(binary_str, 2)

In [195]:
ts = apply_mask_to_num('349380', mask.groups()[0])

tsl = list(ts)

x_pos = [k for k,v in dict(enumerate(ts)).items() if v == "X"]

In [198]:
x_pos

[3, 7, 17, 27]

In [199]:
x_pos

n = len(x_pos)

for repl in list(product("10", repeat=n)):
    
    new_tsl = tsl.copy()
    
    for i in range(n):
        new_tsl[x_pos[i]] = repl[i]
        
    print(int("".join(new_tsl), 2))

41255132631
41255132375
41254870487
41254870231
40986697175
40986696919
40986435031
40986434775
36960165335
36960165079
36959903191
36959902935
36691729879
36691729623
36691467735
36691467479


In [173]:
def generate_memory_addresses(orig_addr: str, mask: str):

    ts = apply_mask_to_num(orig_addr, mask)

    tsl = list(ts)

    x_pos = [k for k, v in dict(enumerate(ts)).items() if v == "X"]

    n = len(x_pos)
    
    for repl in list(product("10", repeat=n)):

        new_tsl = tsl.copy()

        for i in range(n):
            new_tsl[x_pos[i]] = repl[i]

        yield int("".join(new_tsl), 2)

In [178]:
# list(generate_memory_addresses("34069", mask.groups()[0]))

In [179]:
memory = {}

In [180]:
def process_group(group: str):
    # mask = re.search(r"mask\s=\s([X|1|0]{36})", group)
    mask = re.search(r"\s=\s([X|1|0]{36})", group)
    
    mask = mask.groups()[0]
    
    instructions = re.findall(r"mem\[(?P<position>\d+)\]\s=\s(?P<value>\d+\b)", group)
    
    for loc, value in instructions:
        
        for new_loc in generate_memory_addresses(loc, mask):
        
            memory[new_loc] = int(value)

In [183]:
for g in groups:
    process_group(g)

In [184]:
sum(memory.values())

4173715962894